# Sequence Model Training
## Metadata Single-Record Embeddings to File

In [11]:
import sys
sys.path.append("../")

import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from modules import utils, models
from tqdm import tqdm

2021-07-22 01:19:47.927567: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [12]:
metadata_train_gen = utils.CombinedDataGen(data_file='../data/master_train_df.csv',
                                           out_mode='meta',
                                           mode='test',
                                           shuffle=False,
                                           scaler_dir='../models/scalers',
                                           include_index=True)
index_map = utils._index_map('../data/master_train_df.csv')
metadata_mdl = tf.keras.models.load_model('../models/metadata_single_rec.h5')

AttributeError: module 'modules.utils' has no attribute '_index_map'

In [ ]:
utils.write_embeddings(generator=metadata_gen, 
                       out_mode='meta',
                       mode='train',
                       model=metadata_mdl,
                       index_map=index_map,
                       save_dir='../data/metadata_sequence_train.pkl',
                       nb_eps=1)

## Train Metadata Sequence Model

In [31]:
def _process_gen_items(tensor_item, mode, out_mode):

    if mode == 'train':
        if out_mode == 'meta':
            ds = tf.data.Dataset.from_tensors(({'meta_0':tensor_item[0]['meta_0'].astype('float32')},
                                               tensor_item[1].astype('float32'))).batch(1)
        if out_mode == 'image':
            ds = tf.data.Dataset.from_tensors(({'img_0':tensor_item[0]['img_0'].astype('float32')},
                                               tensor_item[1].astype('float32')))
            ds = ds.map(utils._process_aug).batch(1)
        if out_mode == 'combined':
            ds = tf.data.Dataset.from_tensors(({'meta_0':tensor_item[0]['meta_0'].astype('float32'),
                                                'img_0':tensor_item[0]['img_0'].astype('float32')},
                                               tensor_item[1].astype('float32')))
            ds = ds.map(utils._process_aug).batch(1)
    else:
        if out_mode=='meta':
            ds = tf.data.Dataset.from_tensors(({'meta_0':tensor_item[0]['meta_0'].astype('float32')},
                                               tensor_item[1].astype('float32'))).batch(1)
        if out_mode == 'image':
            ds = tf.data.Dataset.from_tensors(({'img_0':tensor_item[0]['img_0'].astype('float32')},
                                               tensor_item[1].astype('float32')))
            ds = ds.map(utils._process).batch(1)
        if out_mode == 'combined':
            ds = tf.data.Dataset.from_tensors(({'meta_0':tensor_item[0]['meta_0'].astype('float32'),
                                                'img_0':tensor_item[0]['img_0'].astype('float32')},
                                               tensor_item[1].astype('float32')))
            ds = ds.map(utils._process).batch(1)
    return ds

def _index_map(csv_file):
    df = pd.read_csv(csv_file)
    
    idx_map = {}
    for row_idx in df.index:
        part_id = df.loc[row_idx, 'participant_id']
        scan_date = df.loc[row_idx, 'scan_date']
        idx_map[row_idx] = [part_id, scan_date]
        
    return idx_map

def write_embeddings(generator, mode, out_mode, model, index_map, 
                     save_dir, nb_eps=1):
    emb_ds = {}
    
    for ep in range(nb_eps):
        print(ep)
        data = {}
        
        for item, r_idx in tqdm(generator):
            r_idx = r_idx[0]
            inp, lbl = item
            ds = _process_gen_items([inp, lbl], mode, out_mode)
            emb = np.squeeze(model.predict(ds)[0])
            sdate, part_id = index_map[r_idx]
            sample = {sdate: [emb, lbl]}
            try:
                data[part_id][sdate] = [emb, lbl]
            except KeyError:
                data[part_id] = sample
        emb_ds[ep] = data
        
    with open(save_dir, 'wb') as handle:
        pickle.dump(emb_ds, handle)

In [ ]:
metadata_seq_mdl = models.init_seq_model(inp_shape=33)
metadata_seq_gen = utils.sequence_generator(data_file='../data/metadata_sequence_train.pkl',
                                            batch_size=64,
                                            emb_shape=33)

In [ ]:
es_cb = callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=1e-8,
                                patience=30,
                                restore_best_weights=True)

metadata_seq_mdl.fit(metadata_seq_gen, 
                     epochs=1000)

In [ ]:
metadata_seq_mdl.save('../models/metadata_sequence_rec.h5')